In [ ]:
import atl08lib
import sys
sys.path.append('/home/pmontesa/code/geoscitools')
import maplib
import pandas as pd
import geopandas as gpd

%matplotlib inline

In [ ]:
import os
os.sys.path.append('/home/pmontesa/code/icesat2')
import FilterUtils

Misc info:
https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb

### After CMR process on DaskHub, insert ADAPT path across list of granules and run extraction (and, optionally, filtering) with shell script
pdsh -g ilab,forest do_extract_filter_atl08.sh \"2018 2019 2020 2021\" /att/nobackup/pmontesa/userfs02/data/icesat2/list_atl08.005 senegal
- Still need to run this

pdsh -g ilab,forest do_extract_filter_atl08.sh \"2018 2019 2020 2021\" /att/nobackup/pmontesa/userfs02/data/icesat2/list_atl08.005 senegal_no_filt

#### Build ATL08 geodataframe from extracted CSVs that were heavily filtered with land-cover specific thresholds for h_can (canopy height)

In [ ]:
main_dir = '/adapt/nobackup/pmontesa/userfs02/data/icesat2/atl08.005/senegal'
atl08_gdf = pd.concat([atl08lib.atl08_io(main_dir, str(YEAR), DO_PICKLE=False) for YEAR in range(2018, 2022)])
atl08_gdf.info()

#### Make an interactive map to view the heavily filtered set of ATL08 obs.
make sure to set SAMP_FRAC so you dont map all the points

In [ ]:
%%time
SAMP_FRAC=0.25
# This is the footprints vector of the ~2m HRSI DSM-derived "CHM" data we are playing with..just for reference
CHM_FOOTPRINTS_FN = '/adapt/nobackup/pmontesa/chm_work/hrsi_chm_senegal/merge.shp'
maplib.MAP_ATL08_FOLIUM(atl08_gdf.sample(frac=SAMP_FRAC), MAP_COL='h_can', DO_NIGHT=False, LAYER_FN = CHM_FOOTPRINTS_FN, RADIUS=3)

# Below here --- old stuff - can delete

#### Read in a set of ATL08 that had no filtering. Make a static map of all the ATL08 from the conservative thresholds

In [ ]:
%%time
main_dir = '/att/gpfsfs/briskfs01/ppl/pmontesa/userfs02/data/icesat2/atl08.005/senegal_no_filt'
atl08_gdf_no_filt = pd.concat([atl08lib.atl08_io(main_dir, str(YEAR), DO_PICKLE=False) for YEAR in range(2018, 2022)])
maplib.MAP_ATL08(atl08_gdf_no_filt, 'senegal','all years', main_dir, YEAR_COL = 'y', proj="+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

### Explore how heavy filtering will decimate the available set of ATL08
#### Run filtering again on top of the unfiltered data, playing with variation in the LC thresholds 
- NOTE : to apply another set of filters on top of previously filtered data, set DO_PREP=False, otherwise it looks for orb_orient and fails....
- well..still fails b/c it wants the orig filter fields ['msw_flg', 'beam_type', 'seg_snow', 'sig_topo'] which get removed after filtering..

In [ ]:
class_values = [0, 111, 113, 112, 114, 115, 116, 121, 123, 122, 124, 125, 126, 20, 30, 90, 100, 60, 40, 50, 70, 80, 200] 
# Aggressive filtering of shrubs, herbaceous - GO WITH THIS
class_thresholds_hcan = [0, 60, 60, 60, 60, 60, 60, 50, 50, 50, 50, 50, 50, 15, 10, 10, 5, 5, 15, 0, 0, 0, 0]
dict_lc_h_can_thresh_TEST2 = dict(zip(class_values, class_thresholds_hcan))

In [ ]:
# atl08_gdf_filt = FilterUtils.filter_atl08_qual_v3(atl08_gdf, SUBSET_COLS=True, DO_PREP=True,
#                                                            subset_cols_list=['rh25','rh50','rh60','rh70','rh75','rh80','rh90','h_can','h_max_can','seg_landcov'], 
#                                                            #subset_cols_list=atl08_cols_list,
#                                                            filt_cols=['h_can','h_dif_ref','m','msw_flg','beam_type','seg_snow','sig_topo'], 
#                                                            dict_lc_h_can_thresh=dict_lc_h_can_thresh_TEST2, thresh_h_dif=25, thresh_sig_topo=2.5, month_min=1, month_max=12)

atl08_gdf_filt = FilterUtils.filter_atl08_qual_v3(atl08_gdf_no_filt, SUBSET_COLS=True, DO_PREP=False,
                                                              subset_cols_list=['rh25','rh50','rh60','rh70','rh75','rh80','rh90','h_can','h_max_can',
                                                                     'h_te_best','granule_name',
                                                                     'seg_landcov','seg_cover','sol_el','y','m','doy'], 
                                                   filt_cols=['h_can','h_dif_ref','m','msw_flg','beam_type','seg_snow','sig_topo'], 
                                                   list_lc_h_can_thresh=class_thresholds_hcan,
                                                   thresh_h_can=100, thresh_h_dif=25, thresh_sig_topo=2.5, month_min=1, month_max=12)

%%time
maplib.MAP_ATL08_FOLIUM(atl08_gdf_filt, MAP_COL='h_can', DO_NIGHT=False, LAYER_FN = '/att/gpfsfs/briskfs01/ppl/pmontesa/chm_work/hrsi_chm_senegal/merge.shp')